<a href="https://colab.research.google.com/github/mahammushtaqdev/Assignment201/blob/main/assigment6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#The code installs libraries for integrating Google GenAI with LangChain to build AI-powered applications.
!pip install -q -U langchain
!pip install -q -U google-genai
!pip install -q -U langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 1.5 MB/s eta 0:00:00


In [3]:
from google.colab import userdata
import os
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Importing ChatGoogleGenerativeAI to interact with Google GenAI
from langchain_google_genai import ChatGoogleGenerativeAI
# Importing Content and Part to handle text content and parts for GenAI
from google.genai.types import Content , Part
# Importing Markdown and display to show results in Jupyter notebooks
from IPython.display import Markdown,display

In [5]:
# Initialize the ChatGoogleGenerativeAI with the specified model and settings
llm = ChatGoogleGenerativeAI(
    model = "gemini-2.0-flash-exp",  # The specific AI model to use
    api_key = userdata.get("GOOGLE_API_KEY"),  # API key for authentication
    temperature = 0.5  # Controls randomness in AI responses (0.0 to 1.0)
)

In [6]:
#Go to the left sidebar > Files > Upload a file

# Step 2: Specify the direct path of the uploaded video
video_file_path = "/content/video.mp4"  #uploaded the video from local storage.

# Step 3: Use the file path
print(f"The video file is accessible at: {video_file_path}")

# Example of accessing the file for further processing
# (You can replace this with your video processing code)
with open(video_file_path, "rb") as video_file:
    print(f"File {video_file_path} is ready for processing.")

The video file is accessible at: /content/video.mp4
File /content/video.mp4 is ready for processing.


In [7]:
#This function uploads a video to Google GenAI and waits for it to finish processing
import time
from google import genai

client = genai.Client(api_key=userdata.get("GOOGLE_API_KEY"))
video_file_path = "/content/video.mp4"


def upload_video(video_file_name):
  video_file = client.files.upload(path = video_file_name)
  while video_file.state == "PROCESSING":
    print("Video Is Being Processed , Kindly Wait!")
    time.sleep(10)
    video_file = client.files.get(name = video_file.name or "")
  if video_file.state == "SUCCESS":
    pass
  elif video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing complete: ' + (video_file.uri or ""))
  return video_file

In [8]:
video_nature = upload_video(video_file_path)

Video Is Being Processed , Kindly Wait!
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/hw4lu0vs6irv


In [9]:
def analyze_video_with_langchain(video_file):
  prompt_template = """For every scene in this video, provide descriptive captions
 summarizing the visuals along with any spoken dialogue enclosed in quotation marks.
 Include each caption in an object, along with its corresponding timecode within the video. """

  response = client.models.generate_content(
    model="gemini-2.0-flash-exp",
    contents=[
        Content(
            role="user",
            parts=[
                Part.from_uri(
                    file_uri=video_file.uri or "",
                    mime_type=video_file.mime_type or ""),
            ]),
        Content(
            role="user",
            parts=[Part(text=prompt_template)]
        )
    ]
  )
  print("Video Analysis:")
  display(Markdown(response.text))

In [10]:
video_1 = video_nature
analyze_video_with_langchain(video_1)

Video Analysis:


```json
[
  {
    "timecode": "0:00",
    "caption": "A leopard and a porcupine are in the center of a road while a car is in the background. Text that reads 'When your ego is bigger than your pain' and 'wild aura' are also displayed."
  },
 {
    "timecode": "0:01",
    "caption":"The leopard makes contact with the porcupine's quills with its paw."
  },
  {
  "timecode": "0:02",
   "caption":"The leopard is examining its paw."
  },
    {
    "timecode":"0:05",
    "caption": "The leopard is still looking at its paw."
  },
    {
      "timecode":"0:08",
      "caption": "The scene changes to black and white while the leopard is trying to remove the quills from its paw."
    },
    {
      "timecode":"0:12",
       "caption":"The scene shows a closeup view of the leopard's face and the porcupine's back in black and white."
     },
  {
    "timecode": "0:14",
    "caption": "The video ends with a closeup of the leopard's face in black and white over a black background with the text 'wild aura' and '@THEREALW.METRA' overlaid on top. An Instagram logo is also displayed."
  }
]
```